In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [78]:
import torch
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch import nn, optim
from tqdm import tqdm
import numpy as np

In [79]:
dataset = load_dataset("itsanmolgupta/mimic-cxr-dataset", split="train")
print(dataset[0])

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7FAD87A07ED0>, 'findings': 'The lungs are clear of focal consolidation, pleural effusion or pneumothorax. The heart size is normal. The mediastinal contours are normal. Multiple surgical clips project over the left breast, and old left rib fractures are noted. ', 'impression': 'No acute cardiopulmonary process.'}


In [80]:
model_name = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)

In [81]:
lora_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=[
        "text_model.encoder.layers.11.self_attn.q_proj",
        "text_model.encoder.layers.11.self_attn.v_proj",
        "vision_model.encoder.layers.11.self_attn.query",
        "vision_model.encoder.layers.11.self_attn.value"
    ]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 16,384 || all params: 151,293,697 || trainable%: 0.0108


In [88]:
named_modules = list(model.named_modules())
for i, (name, module) in enumerate(named_modules):
    print(f"{i:3d}: {name}")

  0: 
  1: base_model
  2: base_model.model
  3: base_model.model.text_model
  4: base_model.model.text_model.embeddings
  5: base_model.model.text_model.embeddings.token_embedding
  6: base_model.model.text_model.embeddings.position_embedding
  7: base_model.model.text_model.encoder
  8: base_model.model.text_model.encoder.layers
  9: base_model.model.text_model.encoder.layers.0
 10: base_model.model.text_model.encoder.layers.0.self_attn
 11: base_model.model.text_model.encoder.layers.0.self_attn.k_proj
 12: base_model.model.text_model.encoder.layers.0.self_attn.v_proj
 13: base_model.model.text_model.encoder.layers.0.self_attn.q_proj
 14: base_model.model.text_model.encoder.layers.0.self_attn.out_proj
 15: base_model.model.text_model.encoder.layers.0.layer_norm1
 16: base_model.model.text_model.encoder.layers.0.mlp
 17: base_model.model.text_model.encoder.layers.0.mlp.activation_fn
 18: base_model.model.text_model.encoder.layers.0.mlp.fc1
 19: base_model.model.text_model.encoder.laye

In [89]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.ds = dataset

    def __getitem__(self, idx):
        item = self.ds[idx]
        return {
            "text": item["findings"],
            "image": item["image"]
        }

    def __len__(self):
        return len(self.ds)

In [91]:
def collate_fn(batch):
    texts = [item["text"] for item in batch]
    images = [item["image"] for item in batch]
    
    inputs = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=77
    )
    return inputs

In [93]:
train_dataset = CLIPDataset(dataset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

def contrastive_loss(logits_per_image, logits_per_text, temperature=0.07):
    batch_size = logits_per_image.shape[0]
    labels = torch.arange(batch_size, device=logits_per_image.device)
    
    logits_per_image = logits_per_image / temperature
    logits_per_text = logits_per_text / temperature
    
    loss_img = F.cross_entropy(logits_per_image, labels)
    loss_txt = F.cross_entropy(logits_per_text, labels)
    
    return (loss_img + loss_txt) / 2

In [94]:
def safe_forward(model, input_ids, attention_mask, pixel_values):
    if hasattr(model, 'base_model'):
        base_model = model.base_model
    else:
        base_model = model
        
    text_outputs = base_model.text_model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    
    vision_outputs = base_model.vision_model(pixel_values=pixel_values)
    text_embeds = text_outputs.pooler_output
    image_embeds = vision_outputs.pooler_output
    text_embeds = base_model.text_projection(text_embeds)
    image_embeds = base_model.visual_projection(image_embeds)

    class CLIPOutput:
        def __init__(self, text_embeds, image_embeds):
            self.text_embeds = text_embeds
            self.image_embeds = image_embeds
    
    return CLIPOutput(text_embeds, image_embeds)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): CLIPModel(
      (text_model): CLIPTextTransformer(
        (embeddings): CLIPTextEmbeddings(
          (token_embedding): Embedding(49408, 512)
          (position_embedding): Embedding(77, 512)
        )
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-10): 11 x CLIPEncoderLayer(
              (self_attn): CLIPSdpaAttention(
                (k_proj): Linear(in_features=512, out_features=512, bias=True)
                (v_proj): Linear(in_features=512, out_features=512, bias=True)
                (q_proj): Linear(in_features=512, out_features=512, bias=True)
                (out_proj): Linear(in_features=512, out_features=512, bias=True)
              )
              (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (mlp): CLIPMLP(
                (activation_fn): QuickGELUActivation()
                (fc1): Linear(in_features=512, out_features=2048, bi

In [95]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
num_epochs = 1

for epoch in range(num_epochs):
    total_loss = 0
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = safe_forward(model, input_ids, attention_mask, pixel_values)
        image_embeds = outputs.image_embeds
        text_embeds = outputs.text_embeds

        image_embeds = F.normalize(image_embeds, p=2, dim=1)
        text_embeds = F.normalize(text_embeds, p=2, dim=1)
        
        logits_per_image = torch.matmul(image_embeds, text_embeds.t())
        logits_per_text = torch.matmul(text_embeds, image_embeds.t())
        
        loss = contrastive_loss(logits_per_image, logits_per_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
        
        if num_batches % 100 == 0:
            avg_loss = total_loss / num_batches
            print(f"Batch {num_batches}, Average Loss: {avg_loss:.4f}")
    
    if num_batches > 0:
        avg_epoch_loss = total_loss / num_batches
        print(f"Epoch {epoch+1} completed. Average Loss: {avg_epoch_loss:.4f}")
    else:
        print(f"Epoch {epoch+1} completed with no successful batches")

Epoch 1/1:   1%|▏         | 102/7659 [00:05<06:53, 18.28it/s]

Batch 100, Average Loss: 1.3993


Epoch 1/1:   3%|▎         | 202/7659 [00:11<06:41, 18.56it/s]

Batch 200, Average Loss: 1.4009


Epoch 1/1:   4%|▍         | 302/7659 [00:16<06:42, 18.27it/s]

...

Batch 7500, Average Loss: 1.3210


Epoch 1/1:  99%|█████████▉| 7602/7659 [06:53<00:03, 18.51it/s]

Batch 7600, Average Loss: 1.3204


Epoch 1/1: 100%|██████████| 7659/7659 [06:56<00:00, 18.38it/s]

Epoch 1 completed. Average Loss: 1.3200


In [98]:
model.save_pretrained("./clip-lora-mimic-cxr")
processor.save_pretrained("./clip-lora-mimic-cxr")

[]

In [101]:
from transformers import CLIPProcessor, CLIPModel
import torch
from peft import PeftModel, PeftConfig
from PIL import Image
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = CLIPProcessor.from_pretrained("./clip-lora-mimic-cxr")
base_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model = PeftModel.from_pretrained(base_model, "./clip-lora-mimic-cxr")
model.to(device)
model.eval()

@torch.no_grad()
def safe_forward(model, input_ids, attention_mask, pixel_values):
    if hasattr(model, 'base_model'):
        base_model = model.base_model
    else:
        base_model = model
    
    text_outputs = base_model.text_model(
        input_ids=input_ids,
        attention_mask=attention_mask
    )
    vision_outputs = base_model.vision_model(pixel_values=pixel_values)
    
    text_embeds = base_model.text_projection(text_outputs.pooler_output)
    image_embeds = base_model.visual_projection(vision_outputs.pooler_output)
    
    return text_embeds, image_embeds

image_path = "/kaggle/input/testing1/test_img1.jpg"
image = Image.open(image_path).convert("RGB")

candidates = [
    "There is evidence of left lower lobe pneumonia.",
    "The heart size is enlarged with signs of pulmonary edema.",
    "Lungs are clear with no acute cardiopulmonary abnormality."
]

inputs = processor(
    text=candidates,
    images=[image] * len(candidates),
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=77
)

inputs = {k: v.to(device) for k, v in inputs.items()}

text_embeds, image_embeds = safe_forward(
    model,
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pixel_values=inputs["pixel_values"]
)

text_embeds = F.normalize(text_embeds, p=2, dim=1)
image_embeds = F.normalize(image_embeds, p=2, dim=1)
similarities = torch.matmul(image_embeds, text_embeds.T)

best_idx = similarities.argmax().item()
print(f"Most similar sentence:\n \"{candidates[best_idx]}\"")
print(f"Similarity scores: {similarities.cpu().numpy()[0]}")

Most similar sentence:
 "Lungs are clear with no acute cardiopulmonary abnormality."
Similarity scores: [0.18905325 0.20127088 0.2176774 ]
